In [1]:
# from datasets import load_dataset, concatenate_datasets

# ted_multi = load_dataset("ted_multi")
# ted_multi = concatenate_datasets([dataset for name, dataset in ted_multi.items()])

In [2]:
# ours = {"ko", "en", "ja", "zh"}

# def func(example, i):
#     count = 0
#     out = dict()
#     lang = example['translations']['language']
#     sentence = example['translations']['translation']
#     print(i)
#     for t in ours:
#         if t in lang:
#             count += 1
#             i = lang.index(t)
#             out[t] = sentence[i]
#         else:
#             continue
    
#     out['count'] = count
#     return out

# ted_m = ted_multi.map(func, with_indices=True)  


In [3]:
import os
import logging
import json
from pathlib import Path

from datasets import load_dataset, DatasetDict

data_dir = Path("/opt/ml/final-project-level3-nlp-01/data/ko-ja")
folder_list = os.listdir(data_dir)

dataset_dict = dict()

for folder in folder_list:
    data = load_dataset("csv", data_files=[str(p) for p in data_dir.joinpath(folder).glob("*.csv")])
    dataset_dict[folder] = data['train']

raw_dataset = DatasetDict(dataset_dict)

Using custom data configuration default-d15fbe40f58f4a20


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /opt/ml/.cache/huggingface/datasets/csv/default-d15fbe40f58f4a20/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-67c6f6653f5a37c1


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /opt/ml/.cache/huggingface/datasets/csv/default-67c6f6653f5a37c1/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
train_set = raw_dataset["Training"]
valid_set = raw_dataset["Validation"]

In [5]:
import random
sample_indices = random.choices(range(len(train_set)), k=5)
sample = train_set.select(sample_indices)
for s in sample:
    print(s, '\n\n')

{'관리번호': 'KO-JA-2020-SOCI-069880', '분야': '사회/노동/복지', '한국어': '도는 각 시·군 판단에 따라 사회적기업이 해당 월 인건비를 우선 청구해 노동자에게 지급할 수 있도록 할 방침이다.', '일본어': '道は各市・郡の判断により、社会的企業が該当月の人件費を先に請求し、労働者に支給できるようにする方針だ。', '한국어_어절수': 17, '일본어_글자수': 47, '길이_분류': 3, '출처': None, '수행기관': '플리토'} 


{'관리번호': 'KO-JA-2020-KPOP-045802', '분야': 'K-POP(한류)/대중문화공연콘텐츠', '한국어': "올여름 가장 뜨거운 논쟁을 불러일으킬 것으로 예상되는 다큐멘터리 '주전장'의 GV 일정이 공개됐다.", '일본어': '今夏、最も熱い論争を引き起こすと予想されるドキュメンタリー「主戦場」のGV日程が公開された。', '한국어_어절수': 12, '일본어_글자수': 39, '길이_분류': 2, '출처': 'https://www.nocutnews.co.kr/news/5187135', '수행기관': '플리토'} 


{'관리번호': 'KO-JA-2020-AUTO-119844', '분야': '자동차/부품/교통/소재', '한국어': '박홍규 SK엔카 사업총괄본부장은 "몇몇 모델의 단종으로 준중형 세단 신차 시장이 축소되면서 중고차 수요가 늘어나고 있다"며 "아반떼의 경우 인기가 많아 잔존가치도 쏘나타나 그랜저보다 높다"고 말했다.', '일본어': 'SKエンカーのパク・ホンギュ事業総括本部長は「いくつかのモデルの生産中止で準中型セダンの新車市場が縮小し、中古車の需要が増えている」とし「アバンテの場合人気が高く、残存価値もソナタやグレンジャーより高い」と述べた。', '한국어_어절수': 24, '일본어_글자수': 95, '길이_분류': 4, '출처': 'https://www.hankyung.com/economy/article/202004214036g', '수행기관': '에버트란'} 


{'관리번호': 'KO-JA-2020

In [6]:
from transformers import AutoModel, AutoTokenizer, MT5Tokenizer, GPT2Model

ENCODER_MODEL = "bert-base-multilingual-cased"
DECODER_MODEL = "THUMT/mGPT"

mBERT = AutoModel.from_pretrained(ENCODER_MODEL)
mBERT_tokenizer = AutoTokenizer.from_pretrained(ENCODER_MODEL)

mGPT = GPT2Model.from_pretrained(DECODER_MODEL)
mGPT_tokenizer = MT5Tokenizer.from_pretrained(DECODER_MODEL)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

/opt/conda/envs/lightweight/lib/python3.7/site-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.07G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'T5Tokenizer'.


In [7]:
mGPT

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(250100, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
    

In [8]:
print(mBERT_tokenizer.tokenize("이순신은조선중기의무신이다."))
print(mBERT_tokenizer.tokenize("아버지가방에들어가신다."))
print(mBERT_tokenizer.tokenize("被酷禍物を加熱したあと、水蒸気を噴射して酷禍させる配置方式の酷禍炉において、水蒸気に含まれた凝縮水などの不純物が、酷禍不良のみならず、脆弱な被酷禍物を損傷させることになる。"))
print()
print(mGPT_tokenizer.tokenize("이순신은조선중기의무신이다."))
print(mGPT_tokenizer.tokenize("아버지가방에들어가신다."))
print(mGPT_tokenizer.tokenize("被酷禍物を加熱したあと、水蒸気を噴射して酷禍させる配置方式の酷禍炉において、水蒸気に含まれた凝縮水などの不純物が、酷禍不良のみならず、脆弱な被酷禍物を損傷させることになる。"))


['이', '##순', '##신', '##은', '##조', '##선', '##중', '##기의', '##무', '##신', '##이다', '.']
['아버지', '##가', '##방', '##에', '##들어', '##가', '##신', '##다', '.']
['被', '酷', '禍', '物', 'を', '加', '熱', 'した', '##あ', '##と', '、', '水', '蒸', '気', 'を', '噴', '射', 'して', '酷', '禍', 'させる', '配', '置', '方', '式', 'の', '酷', '禍', '炉', 'において', '、', '水', '蒸', '気', 'に', '含', 'まれた', '凝', '縮', '水', 'などの', '不', '純', '物', 'が', '、', '酷', '禍', '不', '良', 'のみ', '##な', '##らず', '、', '脆', '弱', 'な', '被', '酷', '禍', '物', 'を', '損', '傷', 'させる', '##ことになる', '。']

['▁이', '순', '신', '은', '조선', '중', '기의', '무', '신', '이다', '.']
['▁아', '버', '지가', '방', '에', '들어', '가', '신', '다', '.']
['▁', '被', '酷', '禍', '物を', '加熱', 'した', 'あと', '、', '水', '蒸', '気を', '噴', '射', 'して', '酷', '禍', 'させる', '配置', '方式', 'の', '酷', '禍', '炉', 'において', '、', '水', '蒸', '気に', '含まれ', 'た', '凝', '縮', '水', 'などの', '不', '純', '物が', '、', '酷', '禍', '不良', 'のみならず', '、', '脆', '弱', 'な', '被', '酷', '禍', '物を', '損', '傷', 'させること', 'になる', '。']


In [9]:
special_tokens_dict = {'additional_special_tokens': ['<s>']}
num_added_toks = mGPT_tokenizer.add_special_tokens(special_tokens_dict)
mGPT.resize_token_embeddings(len(mGPT_tokenizer))
mGPT_tokenizer.bos_token = "<s>"

In [14]:
def func(examples, max_length, max_target_length):

    model_inputs = mBERT_tokenizer(examples['한국어'], max_length=max_length, padding="max_length", truncation=True)
    
    target_sentences = [mGPT_tokenizer.bos_token + ex for ex in examples['일본어']]
    decoder_inputs = mGPT_tokenizer(target_sentences, max_length=max_target_length, padding="max_length", truncation=True)

    model_inputs['decoder_input_ids'] = decoder_inputs['input_ids']
    model_inputs['decoder_attention_mask'] = decoder_inputs['attention_mask']
    model_inputs['labels'] = [ex[1:] for ex in decoder_inputs['input_ids']]

    return model_inputs

fn_kwargs = {"max_length": 512, "max_target_length": 1024}
tokenized_train = train_set.map(func, num_proc=5, batched=True, remove_columns=train_set.column_names, fn_kwargs=fn_kwargs)
tokenized_valid = valid_set.map(func, num_proc=5, batched=True, remove_columns=valid_set.column_names, fn_kwargs=fn_kwargs)


In [16]:
import random
sample_indices = random.choices(range(len(tokenized_valid)), k=1)

sample = tokenized_train.select(sample_indices)
for s in sample:
    print(f"inputs: {mBERT_tokenizer.decode(s['input_ids'])}")
    print(f"decoder inputs: {mGPT_tokenizer.decode(s['decoder_input_ids'])}")
    print(f"labels: {mGPT_tokenizer.decode(s['labels'])}", "\n\n")

    print(s)


inputs: [CLS] 따라서, 본 발명에 따르면 발광 장치의 휘도와 색재현성을 향상시킬 수 있다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [17]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

train_dataloader = DataLoader(
    tokenized_train, batch_size=4, pin_memory=True, shuffle=True, 
    drop_last=True, num_workers=5, collate_fn=default_data_collator,
)

valid_dataloader = DataLoader(
    tokenized_valid, batch_size=4, pin_memory=True, shuffle=False, 
    drop_last=True, num_workers=5, collate_fn=default_data_collator,
)

In [18]:
train_batch = next(iter(train_dataloader))
valid_batch = next(iter(valid_dataloader))

In [19]:
d_inputs = train_batch.pop("decoder_input_ids")
d_attn_mask = train_batch.pop("decoder_attention_mask")
labels = train_batch.pop("labels")

encoder_outputs = mBERT(**train_batch)
decoder_outputs = mGPT(input_ids=d_inputs, attention_mask=d_attn_mask)

In [22]:
encoder_outputs.last_hidden_state.shape

torch.Size([4, 512, 768])

In [23]:
decoder_outputs.logits.shape

torch.Size([4, 1024, 250101])